In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [2]:
e_eff=(11.4+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [3]:
import pandas as pd

# Q1 related parameters

In [4]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [5]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [8]:
path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'

In [9]:
data=pd.read_csv(path+'\Q1fine_sim.csv',sep=';')# matrix should be done on pF

In [10]:
data

,Unnamed: 0,Charge,ground,Q1bottom,Q1top,resonator
0,Charge,0.006063,-0.005913,-0.000032,-0.000117,-0.000002
1,ground,-0.005913,0.199550,-0.083040,-0.090835,-0.019762
2,Q1bottom,-0.000032,-0.083040,0.129420,-0.039168,-0.007184
3,Q1top,-0.000117,-0.090835,-0.039168,0.130870,-0.000747
4,resonator,-0.000002,-0.019762,-0.007184,-0.000747,0.027694


In [11]:
q_name='Q1'
Q1_cap_ground_top=-data[q_name+'top'][1]
Q1_cap_ground_bottom=-data[q_name+'bottom'][1]
Q1_cap_top_bottom=-data[q_name+'bottom'][3]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [12]:
Ec(Q1_cap)

234.7

### Resonator coupling

In [66]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc+Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [70]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,1))# return in fF

In [15]:
Q1_cap_res_top=-data[q_name+'top'][4]
Q1_cap_res_bottom=-data[q_name+'bottom'][4]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [16]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

0.7
7.2
3.9


In [39]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [41]:
Zr=50
omega_r=5.5
omega_q=4.5
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

44.7

### Dispersive shift

In [48]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [64]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.3798102559326152

### Readout resonator properties

In [91]:
Q=10000
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,2)# first value in ns

(289.4, 0.55)

In [94]:
T=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
T*1e6#us

144.82461941326272

### Charge line

In [72]:
Q1_cap_charge_line_top=-data[q_name+'top'][0]
Q1_cap_charge_line_bottom=-data[q_name+'bottom'][0]
Cc_charge=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_charge_line_top,Q1_cap_charge_line_bottom)
cap_pretty_print([Q1_cap_charge_line_top,Q1_cap_charge_line_bottom,Cc_charge],1e6)#in aF

116.5
31.6
72.1


In [84]:
Z0=50
T1=(Cc_charge+Q1_cap)/(Z0*(2*np.pi*omega_q*Cc_charge)**2)*1e-6

In [88]:
T1*1e6#in us

397.6797739128156

### Coupling between qubits